This project is aimed at helping businesses streamline their invoice management process by automating reminders and notifications to customers with overdue invoices. The Python programming language is used to connect to the Stripe API and retrieve information about open invoices, which is then stored in a MySQL database. 

The project provides a simple and effective solution for businesses looking to improve their cash flow by ensuring timely payment of invoices. The use of automation reduces manual workload and allows businesses to focus on other important tasks. With the ability to customize email templates and set up automatic reminders, the project provides flexibility and ease of use.

In [6]:
#pip install stripe --upgrade
#pip install scraper-api --upgrade


ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.10,<4.0; 0.1.1b0 Requires-Python >=3.10,<4.0
ERROR: Could not find a version that satisfies the requirement scraper-api (from versions: none)
ERROR: No matching distribution found for scraper-api


In [7]:
class Config:
    def __init__(self):
        self.host = ""
        self.database = ""
        self.db_user = ""
        self.db_password = ""

obj_config = Config()

In [11]:
from collections import Counter
import io
import json
import os
import re
import time
from datetime import datetime
import pandas as pd
import stripe

stripe.api_key = ""

In [9]:
def get_connection():
    """
    Connects to MySQL Database
    :return:
    """
    return mysql.connector.connect(
        host = obj_config.host,
        database = obj_config.database,
        user = obj_config.db_user,
        password = obj_config.db_password
        )

In [ ]:
def start_display(app_name, start_time):
    """
    Application start display
    :param app_name:
    :param start_time:
    :return:
    """
    print("#" * 100 + "\n")
    title = app_name.center(100)
    print(title + "\n")
    beautify(SPACE, "Date::Time", ": {}".format(datetime.strftime(start_time, "%Y/%m/%d::%H-%M-%S")))
    print("\n" + "#" * 100 + "\n")
    return start_time


def end_display(start_time):
    """
    Application end display
    :param start_time: start_time for application
    :return:
    """
    print("\n" + "#" * 100 + "\n")
    end_time = datetime.now()
    beautify(
        SPACE, "Completed Task at", ": {}".format(datetime.strftime(end_time, "%Y/%m/%d::%H-%M-%S"))
        )
    time_duration = (end_time - start_time)
    hours = int(time_duration.seconds // (60 * 60))
    minutes = int((time_duration.seconds // 60) % 60)
    beautify(SPACE, "Time taken to complete the task", ": {} hour(s) {} minute(s)".format(hours, minutes))
    print("\n" + "#" * 100)

In [ ]:
def escape_name(s):
    """
    Escaping strings
    :param s:
    :return: string
    """
    return '`{}`'.format(s.replace('`', '``'))


def beautify(space, *arg_strings):
    """
    Aligns the output with the given space

    :param arg_strings: string arguments to be printed
    :param space: defines number of spaces between strings
    :return: evenly spaced single string for all the input strings
    """
    count = 1
    text = arg_strings[0]
    for strn in arg_strings[1:]:
        text = text + " " * ((count * space) - len(text)) + strn
        count += 1
    print(text)

In [10]:
def insert_into_open_invoices(values):
    """
    Inserts invoice data into mysql table
    :param values:
    :return:
    """
    connection = get_connection()
    cursor = connection.cursor(buffered = True)
    names = list(values[0])
    cols = ", ".join(map(escape_name, names))
    placeholders = ", ".join(['%({})s'.format(name) for name in names])
    query = 'INSERT INTO {} ({}) VALUES ({}) on duplicate key update ' \
            'amount_remaining = VALUES(amount_remaining), ' \
            'invoice_pdf = VALUES(invoice_pdf), ' \
            'hosted_invoice_url = VALUES(hosted_invoice_url),' \
            'tax = VALUES(tax)'\
        .format('open_invoices', cols, placeholders)
    cursor.executemany(query, values)
    connection.commit()
    cursor.close()
    connection.close()


def get_existing_open_invoices():
    connection = get_connection()
    try:
        cursor = connection.cursor(buffered = True)
        query = 'SELECT invoice_id FROM open_invoices'
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()
        connection.close()
        return result
    except Exception as e:
        beautify(SPACE, "Error", ": " + str(e))

In [ ]:
old_invoices = get_existing_open_invoices()

flag = True

invoices = []

temp = stripe.Invoice.list(status = "open", limit = 100)
for x in temp:
    invoices.append(x)

while flag:
    temp = stripe.Invoice.list(status = "open", limit = 100, starting_after = invoices[len(invoices) - 1])
    if len(temp) < 1:
        flag = False
    else:
        for x in temp:
            invoices.append(x)

open_invoices = []
invoice_ids = []

for open_invoice in invoices:
    if open_invoice["due_date"] is not None:
        due_date = datetime.utcfromtimestamp(open_invoice["due_date"]).strftime('%Y-%m-%d %H:%M:%S')
    else:
        due_date = datetime.utcfromtimestamp(open_invoice["created"]).strftime('%Y-%m-%d %H:%M:%S')
    description = []
    for i in range(0, len(open_invoice["lines"]["data"])):
        description.append(open_invoice["lines"]["data"][i]["description"].encode('utf-8', errors = 'ignore').decode('utf-8'))
    memo = ""
    if open_invoice["description"] is not None:
        memo = open_invoice["description"]
        if "CARIFY - CLAIMS" in open_invoice["description"]:
            carify_claim = True
        else:
            carify_claim = False
    else:
        carify_claim = False
    if open_invoice["tax"] is None:
        tax = 0.0
    else:
        tax = float(open_invoice["tax"]) / 100
    invoice = {
        "invoice_id": open_invoice["id"],
        "invoice_link": "https://dashboard.stripe.com/invoices/" + open_invoice["id"],
        "amount_due": float(open_invoice['amount_due']) / 100,
        "amount_remaining": float(open_invoice["amount_remaining"]) / 100,
        "tax": str(tax),
        "billing_reason": open_invoice["billing_reason"],
        "collection_method": open_invoice["collection_method"],
        "created": datetime.utcfromtimestamp(open_invoice["created"]).strftime('%Y-%m-%d %H:%M:%S'),
        "customer": open_invoice["customer"],
        "customer_email": open_invoice["customer_email"],
        "customer_name": open_invoice["customer_name"] if open_invoice["customer_name"] is not None else "",
        "customer_phone": open_invoice["customer_phone"] if open_invoice["customer_phone"] is not None else "",
        "street": open_invoice["customer_address"]["line1"] if open_invoice["customer_address"]["line1"] is not None else "",
        "city": open_invoice["customer_address"]["city"] if open_invoice["customer_address"]["city"] is not None else "",
        "zip": open_invoice["customer_address"]["postal_code"] if open_invoice["customer_address"]["postal_code"] is not None else "",
        "invoice_pdf": open_invoice["invoice_pdf"],
        "hosted_invoice_url": open_invoice["hosted_invoice_url"],
        "subscription": open_invoice["subscription"],
        "next_payment_attempt": datetime.utcfromtimestamp(open_invoice["next_payment_attempt"]).strftime('%Y-%m-%d %H:%M:%S') if open_invoice["next_payment_attempt"] is not None else "",
        "description": str(description).replace("[","").replace("]",""),
        "due_date": due_date,
        "carify_claim": carify_claim,
        "memo": memo,
        "invoice_number": open_invoice["number"],
        "invoice_items": open_invoice["lines"]["total_count"],
        "payment_intent": open_invoice["payment_intent"],
        "period_end": datetime.utcfromtimestamp(open_invoice["period_end"]).strftime('%Y-%m-%d %H:%M:%S'),
        "period_start": datetime.utcfromtimestamp(open_invoice["period_start"]).strftime('%Y-%m-%d %H:%M:%S'),
        }
    open_invoices.append(invoice)
    invoice_ids.append(invoice["invoice_id"])

insert_into_open_invoices(open_invoices)

existing_invoices = pd.DataFrame(old_invoices, columns = ['invoice_id'])['invoice_id'].tolist()

cleared_invoices_list = list((Counter(existing_invoices) - Counter(invoice_ids)).elements())

if len(cleared_invoices_list) > 0:
    beautify(SPACE, "Updating cleared invoices in mysql", ": " + str(len(cleared_invoices_list)))
    connection = get_connection()
    cursor = connection.cursor(buffered = True)
    for i in range(0, len(cleared_invoices_list)):
        query_string = "delete from open_invoices where invoice_id = '{}'".format(cleared_invoices_list[i])
        cursor.execute(query_string)
        connection.commit()
    cursor.close()
    connection.close()